In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
# import torchvision
# import torchvision.transforms as transforms

In [3]:
print(torch.__version__)

2.0.0+cu118


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Define File Path
vec20avg_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec20_avg.npz"
vec25avg_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec25_avg.npz"
vec30avg_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec30_avg.npz"
vec35avg_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec35_avg.npz"
vec20sum_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec20_sum.npz"
vec20sum_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec20_sum.npz"
vec25sum_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec25_sum.npz"
vec30sum_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec30_sum.npz"
vec35sum_path = "/content/drive/MyDrive/Co-working /Thesis/vec_data/vec35_sum.npz"
freq_stance_labels = "/content/drive/MyDrive/Co-working /Thesis/vec_data/freq_stance_labels.npz"
oh_stance_labels = "/content/drive/MyDrive/Co-working /Thesis/vec_data/oh_stance_labels.npz"

In [6]:
def load_npz_file(filepath):
    # Load the numpy array from the .npz file
    with np.load(filepath, allow_pickle=True) as data:
        for key in data.keys():
            arr = data[key]
            break
    return arr

In [7]:
vec20avg = load_npz_file(vec20avg_path)
vec25avg = load_npz_file(vec25avg_path)
vec30avg = load_npz_file(vec30avg_path)
vec35avg = load_npz_file(vec35avg_path)
vec20sum = load_npz_file(vec20sum_path)
vec20sum = load_npz_file(vec20sum_path)
vec25sum = load_npz_file(vec25sum_path)
vec30sum = load_npz_file(vec30sum_path)
vec35sum = load_npz_file(vec35sum_path)
freq_label = load_npz_file(freq_stance_labels)
oh_label = load_npz_file(oh_stance_labels)

In [227]:
#指派實際要使用的Data與Label
data = vec30avg
label = np.argmax(oh_label, axis=1)
# label = freq_label
print(data.shape)
print(label.shape)

(445, 1, 300)
(445,)


In [228]:
print(label)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 0 0 1
 1 2 1 1 1 1 1 1 1 1 1 1 2 2 2 1 2 1 2 1 1 2 2 2 1 1 1 0 0 0 2 2 1 1 1 1 1
 1 1 1 1 1 2 1 1 1 2 2 1 2 1 2 1 2 1 2 2 1 2 2 2 1 1 2 1 1 1 1 1 2 1 1 2 2
 1 1 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 2 2 1 2 1 1 2 1
 2 1 1 1 1 1 1 2 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 2 2
 1 1 2 1 1 1 2 2 2 2 1 1 2 2 1 2 2 2 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 0
 0 0 0 0 0 1 2 2 1 1 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 2 1 1 1
 2 1 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


In [229]:
train_data, test_data, train_labels, test_labels = train_test_split(data, label, test_size=0.2)
print('Train data shape:', train_data.shape)
print('Train labels shape:', train_labels.shape)
print('Test data shape:', test_data.shape)
print('Test labels shape:', test_labels.shape)

Train data shape: (356, 1, 300)
Train labels shape: (356,)
Test data shape: (89, 1, 300)
Test labels shape: (89,)


In [230]:
print(train_data[0])

[[-7.08233416e-02 -6.28900081e-02  3.91806632e-01  1.55733332e-01
   8.57666414e-03  4.06366624e-02  4.04233299e-02 -5.65500036e-02
   7.58366734e-02  1.15633318e-02  1.82499997e-02  6.20866641e-02
  -3.12599987e-02  9.91666783e-03  8.74333363e-03 -2.41999817e-03
   1.15166660e-02  2.26600040e-02 -3.59333167e-03  2.36233342e-02
   8.41666386e-03 -1.17499996e-02  4.41100001e-02  2.83900034e-02
   1.37300007e-02  3.12699974e-02 -3.10600046e-02 -1.21400002e-02
   3.49766687e-02 -1.57866664e-02  6.04166612e-02 -1.89566687e-02
   7.47766718e-02 -2.58799959e-02 -8.63566697e-02 -3.96566652e-02
  -1.54866660e-02 -1.53846681e-01 -6.99666794e-03  4.81000356e-03
   2.93199997e-02  4.32400033e-02 -4.33766730e-02  3.05566657e-02
   5.84566630e-02  8.23000353e-03 -4.21976686e-01 -1.17333385e-03
   3.34033370e-02 -8.78333859e-03  2.88866702e-02 -4.97166626e-02
  -2.36333325e-03 -1.69893339e-01 -4.79966626e-02  3.36333271e-03
   7.44600073e-02 -2.48433352e-02 -3.93966697e-02  1.02799991e-02
  -5.94599

In [231]:
print(train_labels[99])

3


In [232]:
# Convert the data and labels to PyTorch tensors
train_data_tensor = torch.from_numpy(train_data).float()
train_labels_tensor = torch.from_numpy(train_labels).float()
test_data_tensor = torch.from_numpy(test_data).float()
test_labels_tensor = torch.from_numpy(test_labels).float()

In [233]:
print(train_labels_tensor)

tensor([1., 0., 2., 2., 2., 2., 3., 3., 1., 1., 3., 2., 1., 1., 1., 3., 1., 0.,
        1., 3., 1., 1., 1., 0., 1., 1., 3., 1., 3., 3., 2., 3., 3., 3., 1., 3.,
        1., 1., 3., 2., 3., 2., 1., 0., 1., 3., 1., 3., 1., 2., 1., 2., 1., 2.,
        2., 1., 1., 1., 3., 2., 1., 3., 3., 3., 3., 1., 3., 2., 1., 2., 3., 3.,
        3., 3., 3., 3., 1., 3., 1., 3., 1., 2., 2., 1., 1., 1., 1., 0., 1., 0.,
        1., 2., 3., 2., 1., 3., 1., 3., 3., 3., 3., 2., 1., 1., 1., 1., 1., 2.,
        3., 1., 2., 3., 3., 3., 1., 3., 3., 3., 3., 1., 2., 3., 1., 2., 2., 3.,
        3., 3., 1., 1., 1., 3., 1., 2., 3., 1., 2., 1., 2., 1., 1., 1., 1., 2.,
        2., 1., 1., 3., 1., 3., 3., 2., 2., 3., 2., 1., 3., 3., 1., 1., 1., 1.,
        1., 2., 3., 1., 1., 1., 3., 3., 2., 1., 3., 1., 1., 3., 3., 1., 3., 1.,
        3., 1., 1., 3., 1., 3., 1., 1., 2., 2., 3., 2., 2., 2., 1., 3., 3., 3.,
        1., 3., 2., 3., 3., 1., 3., 3., 1., 3., 3., 2., 3., 3., 3., 2., 1., 1.,
        3., 3., 1., 3., 1., 3., 1., 1., 

In [234]:
# Create a TensorDataset for each set of data and labels
train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)

In [235]:
# Create a DataLoader for each dataset to batch and shuffle the data
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [244]:
# Define the CNN model architecture
# class CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool1d(kernel_size=2)
#         self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool1d(kernel_size=2)
#         self.fc1 = nn.Linear(in_features=2400, out_features=64)
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(in_features=64, out_features=num_classes)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
#         x = self.pool2(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         return x

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=5)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 73, 256)
        self.fc2 = nn.Linear(256, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        print(x.shape)
        x = x.view(-1, 64 * 73)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [245]:
def evaluate_loss(model, loader, criterion):
    total_loss = 0.0
    total_samples = 0

    for inputs, labels in loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

    return total_loss / total_samples

In [246]:
# Create an instance of the CNN model
num_classes = 4
model = CNN()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Define the loss function and optimizer
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [247]:
# Define the number of epochs to wait before stopping if the test accuracy does not improve
patience = 3
# Initialize the counter for the number of epochs without improvement
counter = 0
# Initialize the best test accuracy to zero
best_val_accuracy = 0.0

In [248]:
# Train the model
num_epochs = 3

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move the inputs and labels to the device
        # inputs = inputs.to(device)
        # labels = labels.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        # Compute the loss
        print(f"Batch {i} labels: {labels}")
        loss = criterion(outputs, labels.argmax(dim=-1))
        # Backward pass and update the parameters
        loss.backward()
        optimizer.step()
        # Update the training loss
        train_loss += loss.item()

    # Compute the training accuracy
    train_predictions = []
    train_true_labels = []
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1)
            train_predictions.extend(predictions.tolist())
            train_true_labels.extend(labels.argmax(dim=-1).tolist())
    train_accuracy = accuracy_score(train_true_labels, train_predictions)

    # Compute the test accuracy
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1)
            val_predictions.extend(predictions.tolist())
            val_true_labels.extend(labels.argmax(dim=-1).tolist())
    val_accuracy = accuracy_score(val_true_labels, val_predictions)

    # Compute the training and test loss
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    val_loss = evaluate_loss(model, test_loader, criterion)
    val_losses.append(val_loss)

    # Record the training and test accuracy
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # Check if the test accuracy has improved
    if val_accuracy > best_val_accuracy:
        # Update the best test accuracy and reset the counter
        best_val_accuracy = val_accuracy
        counter = 0
    else:
        # Increment the counter
        counter += 1
        # Check if the counter has reached the patience limit
        if counter == patience:
            print(f"Early stopping after {epoch+1} epochs.")
            break

    # Print the epoch number, training loss, training accuracy, and test accuracy
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f} - Test Accuracy: {val_accuracy:.4f} - Test Loss: {val_loss:.4f}")


torch.Size([32, 64, 73])
Batch 0 labels: tensor([3., 3., 2., 0., 1., 1., 2., 3., 1., 0., 1., 1., 3., 3., 2., 1., 3., 3.,
        3., 1., 3., 1., 3., 1., 1., 1., 3., 2., 1., 1., 1., 2.])


ValueError: ignored

In [ ]:
# Plot the train and test accuracy and loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(train_accuracies, label='Train')
ax1.plot(val_accuracies, label='Test')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

ax2.plot(train_losses, label='Train')
ax2.plot(val_losses, label='Test')
ax2.set_title('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()